In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender


#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [3]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')



#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

In [11]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=1000, regularization=0.001,iterations=2)

# train the model on a sparse matrix of item/user/confidence weights
item_user_data=URM_train.T
print("ITEM USER ={}".format(item_user_data.shape))
model.fit(item_user_data)

# recommend items for a user
user_items = item_user_data.T.tocsr()
print("USER ITEM ={}".format(user_items.shape))


#uno per volta predico tutti gli utenti e estraggo dalla lista delle 10 predizioni, dalla tupla, la track
rec_matrix=np.zeros((50446,10))

for u in all_userID:
    rec = model.recommend(u, user_items)
    for x in range (10):
        rec_matrix[u,x]=rec[x][0]
print(rec_matrix)   

ITEM USER =(20635, 50446)


100%|██████████| 2.0/2 [01:33<00:00, 49.05s/it]


USER ITEM =(50446, 20635)
[[9.3260e+03 2.0268e+04 5.4100e+02 ... 1.7203e+04 1.2436e+04 2.5080e+03]
 [9.3260e+03 7.0090e+03 2.3170e+03 ... 1.4453e+04 1.6601e+04 1.3216e+04]
 [4.0000e+00 1.7586e+04 1.2173e+04 ... 1.3980e+04 2.6740e+03 1.8026e+04]
 ...
 [9.3260e+03 4.9280e+03 8.8850e+03 ... 1.8840e+04 2.2900e+02 8.4430e+03]
 [4.0000e+00 1.6173e+04 1.7240e+03 ... 1.7780e+04 6.6000e+02 9.4120e+03]
 [4.0000e+00 3.5650e+03 7.0350e+03 ... 1.3259e+04 1.0251e+04 1.1006e+04]]


In [12]:
print(model.fit(item_user_data))

100%|██████████| 2.0/2 [01:12<00:00, 37.62s/it]

None


In [13]:
import numpy as np
import scipy.sparse as sps



def precision(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score



def recall(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score



def MAP(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

In [14]:
def evaluate_algorithm(URM_test, recMatrix, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0

    num_eval = 0

    URM_test = sps.csr_matrix(URM_test)

    n_users = URM_test.shape[0]


    for user_id in range(n_users):

        if user_id % 10000 == 0:
            print("Evaluated user {} of {}".format(user_id, n_users))

        start_pos = URM_test.indptr[user_id]
        end_pos = URM_test.indptr[user_id+1]

        if end_pos-start_pos>0:

            relevant_items = URM_test.indices[start_pos:end_pos]

            recommended_items = recMatrix[user_id]
            num_eval+=1

            is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

            cumulative_precision += precision(is_relevant, relevant_items)
            cumulative_recall += recall(is_relevant, relevant_items)
            cumulative_MAP += MAP(is_relevant, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval

    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP))

    result_dict = {
        "precision": cumulative_precision,
        "recall": cumulative_recall,
        "MAP": cumulative_MAP,
    }

    return result_dict

In [15]:
evaluate_algorithm(URM_test,rec_matrix)

Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0597, Recall = 0.1390, MAP = 0.0499


{'MAP': 0.04990441831789887,
 'precision': 0.059725093013644856,
 'recall': 0.13900930312953222}

In [16]:
#Salviamom la matrice con i ranking per poterlo ibridare in futuro
colonne= 20635
rec_matrix=np.zeros((50446,colonne))

for u in all_userID:
    if (u==1000):
        print("Blocco")
    if (u==5000):
        print("BloccoX")
    if (u==10000):
        print("Blocco1")
    if (u==20000):
        print("Blocco2")
    if (u==30000):
        print("Blocco3")
    if (u==40000):
        print("Blocco4")
    if (u==50000):
        print("Blocco5")
    rec = model.recommend(u, user_items,N=colonne)
    for x in range (len(rec)):
        col=rec[x][0]
        rec_matrix[u,col]=rec[x][1]
print(rec_matrix)  

Blocco
BloccoX
Blocco1
Blocco2
Blocco3
Blocco4
Blocco5
[[           nan            nan 1.04453344e+01 ...            nan
             nan            nan]
 [           nan            nan 3.53467070e+04 ...            nan
             nan            nan]
 [           nan            nan 2.29847780e+07 ...            nan
             nan            nan]
 ...
 [           nan            nan 2.05020867e-02 ...            nan
             nan            nan]
 [           nan            nan 1.84009187e-02 ...            nan
             nan            nan]
 [           nan            nan 2.61278488e-02 ...            nan
             nan            nan]]


In [4]:
from KNN.ScoresHybridRecommenderXXX import ScoresHybridRecommenderXXX
SVDrecommender = PureSVDRecommender (URM_train)
SVDrecommender.fit()

PureSVDRecommender Computing SVD decomposition...
PureSVDRecommender Computing SVD decomposition... Done!


NameError: name 'rec_matrix' is not defined

In [ ]:
Hrecommender = ScoresHybridRecommenderXXX(URM_train,SVDrecommender,rec_matrix)
Hrecommender.fit(0.4)

In [ ]:
result = Hrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

In [ ]:
alpha= [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for a in alpha:
    print(a)
    Hrecommender.fit(alpha=a)
    result = Hrecommender.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))